# Train Basic ML Agent for Connect 4

This notebook trains a simple machine learning model to predict the best move from a given Connect 4 board state.  
It uses the UCI Connect 4 dataset and outputs `ml_agent.pkl`, which is used as the basic ML agent in the game.

## 1. Define Move Label Function

Function to extract the first available move from a board state.

In [4]:
def find_first_available_column(row):
    for col in range(7):
        for row_idx in range(5, -1, -1):  # from bottom to top
            cell_index = row_idx * 7 + col
            if row[cell_index] == 'b':  # found an empty cell
                return col  # 1st playable column
    return None  # no moves available

## 2. Setup

Import required Python libraries.

In [6]:
import pandas as pd

## 3. Load Dataset

Load the Connect 4 dataset.

Dataset Reference: https://archive.ics.uci.edu/dataset/26/connect+4

In [8]:
df = pd.read_csv("connect-4.data", header=None)

In [9]:
display(df)

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
1,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
2,b,b,b,b,b,b,o,b,b,b,...,b,b,b,b,b,b,b,b,b,win
3,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
4,o,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67552,x,x,b,b,b,b,o,x,o,b,...,b,b,b,o,o,x,b,b,b,loss
67553,x,x,b,b,b,b,o,b,b,b,...,b,b,b,o,x,o,o,x,b,draw
67554,x,x,b,b,b,b,o,o,b,b,...,b,b,b,o,x,x,o,b,b,loss
67555,x,o,b,b,b,b,o,b,b,b,...,b,b,b,o,x,o,x,x,b,draw


## 4. Label Data

Apply the move label function to each row to generate target labels.

In [11]:
df["move"] = df.iloc[:, 0:42].apply(find_first_available_column, axis=1)

# Remove full board positions where there's no valid move
df = df[df["move"].notna()]

## 5. Convert Board Symbols to Numeric

Convert board symbols to numbers so the model can understand the input.

In [ ]:
# x -> 1 (Player 1)
# o -> -1 (Player 2)
# b = 0 (empty)
df.iloc[:, 0:42] = df.iloc[:, 0:42].replace({'x': 1, 'o': -1, 'b': 0})

In [14]:
display(df)

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,move
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,win,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,win,0
2,0,0,0,0,0,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,win,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,win,0
4,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,win,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67552,1,1,0,0,0,0,-1,1,-1,0,...,0,0,-1,-1,1,0,0,0,loss,0
67553,1,1,0,0,0,0,-1,0,0,0,...,0,0,-1,1,-1,-1,1,0,draw,0
67554,1,1,0,0,0,0,-1,-1,0,0,...,0,0,-1,1,1,-1,0,0,loss,0
67555,1,-1,0,0,0,0,-1,0,0,0,...,0,0,-1,1,-1,1,1,0,draw,0


## 6. Split Dataset

Split the data into training and test sets.

In [16]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, 0:42]  # board
y = df["move"]  # best move column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 7. Train Model

Train a classifier on the processed dataset.

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

model = DecisionTreeClassifier()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))

Accuracy: 1.0


## 8. Save Trained Model

Export the trained model as `ml_agent.pkl`.

In [20]:
import joblib
joblib.dump(model, "ml_agent.pkl")

['ml_agent.pkl']